# QUESTION 2

Import below in order to play functions

In [4]:
import pygame 
import numpy as np
import matplotlib.pyplot as plt

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Generating tree data

The function below generates data for a tree which is stored as two lists of branches. The first list, trunks represents the trunks of every new generated tree and the second list takes only the end branches. Since every branch in the recursive tree is either an end branch or the trunk of another tree then those two lists suffice.Each value in the trunks list is given by [[[x0,y0],[x1,y1]],d] where (x0,y0) are the start points of the trunk, (x1,y1) are the end points of the branch and d is the thickness of the trunk. This allows us to build a tree with more structure. 

There is a reference tree at the end of the notebook

In [5]:
def make_tree(depth, branch1, branch_list_end,trunks,
                  alpha,beta, rat1 = 0.5, rat2 = 1):
    ''' function imputs: depth, branch1, branch_list_end,branch_list
    alpha
    function recursively generates branches of a tree with 
    depth takes the depth of tree with a tree with 2 branches having depth 1
    and a trunk having depth 0
    branch1 takes the x and y coordinates of the trunk of the tree in the form 
    [[x0,y0],[x1,y1]] 
    branch_list_end is the list to be modified with the end branches of the tree
    trunks is the list to be modified with the non-end branches of the tree
    alpha is the angle the branches make with the trunk in radians
    beta is the inital angle made with the trunk
    rat1 takes a float between 0 and 1 and interprets that as the 
    distance from point 0 to 2 divided by distance from 0 to 1
    rat2 takes a float greater that 0 and interprets that as the ratio in 
    length between the central branch and the other two 
    (ie if rat2 = 1 they are the same length, if rat2 = 0.5 the auxillary branches 
     are half the length of the middle branch
    '''
    
    # Initiate by setting the start and end points of the first branch to variables
    (x0,y0) = branch1[0]
    (x1,y1) = branch1[1]
    
    # Base case
    # When the depth is zero then the branch is an end branch. 
    if depth == 0:
        branch_list_end.append(branch1)
        return None
    
    else:
        # Here calculate the branches 
        length = np.sqrt((x1 - x0)**2 + (y1 - y0)**2)
        # length is the length of the branch given. 
        auxbran = (1-rat1)*rat2*length
        # auxbran is the auxillary branch length
        x2 = (x1-x0)*(rat1) + x0
        y2 = (y1-y0)*(rat1) + y0
        # (x2,y2) is the point from which the branches originate
        x3 = auxbran*np.cos(beta - alpha) + x2
        y3 = auxbran*np.sin(beta - alpha) + y2
        # (x3,y3) is the end point of the right branch
        x4 = auxbran*np.cos(beta + alpha) + x2
        y4 = auxbran*np.sin(beta + alpha) + y2
         # (x4,y4) is the end point of the left branch
            
        # Then call the function recursively in order to generate branches on those branches
        # Middle branch (between points 2 and 1)
        # Here beta stays the same as before as the angle between the branch the floor is the same
        branch2 = [[x2,y2],[x1,y1]]
        make_tree(depth-1, branch2, branch_list_end, 
                      trunks, alpha, beta)
        # Right branch (between points 2 and 3)
        # angle between floor and branch is beta - alpha
        trunks.append([[[x0,y0],[x2,y2]],depth])
        branch3 = [[x2,y2],[x3,y3]]
        make_tree(depth-1, branch3, branch_list_end, 
                      trunks, alpha, beta - alpha )
        # Left branch (between points 2 and 4)
        # angle between floor and branch is beta + alpha
        branch4 = [[x2,y2],[x4,y4]]
        make_tree(depth-1, branch4, branch_list_end, trunks, 
                      alpha, beta + alpha )
        
        return None 
    

Then here is the function to draw the trees

In [6]:
def draw_tree(depth = 7,alpha = np.pi/6, rat1 = 0.5, rat2 = 1, 
                  speed_factor = 3, thickness = False):
    '''
    Function that draws tree as an animation. 
    The depth of the tree (recursion) can be adjusted by entering 
    a depth integer value as a parameter. 
    rat1 takes a float between 0 and 1 and interprets that as the 

    rrat2 takes a float greater that 0 and interprets thats as the ratio in 
    length between the central branch and the other two 
    (ie if rat2 = 1 they are the same length, if rat2 = 0.5 the central branches 
     are half the length of the middle branch)
    speed_factor is speed factor
    thickness is true when depth can be indicated on the image using width of branches
    '''
    # Initialise variables
    dimensions = (900, 700)
    backgroundColour = (255,255,255)
    brown, green = (92, 52, 13), (6, 66, 11)
    # Here is the first branch(trunk) of the tree
    trunk = [[450,500],[450,50]]
    min_depth, max_depth = 1, 10
    clock = pygame.time.Clock()
    warning = "Depth must be an integer in the interval [1,10]"

    if depth < min_depth: 
        depth = min_depth
        print(warning)
        print("Using depth {}".format(min_depth))
    if depth > max_depth: 
        depth = max_depth
        print(warning)
        print("Using depth {}".format(max_depth))

    frames_per_second = 20  + 10 * speed_factor
    # Here are lists to be altered by function make_tree
    branch_list_end = []
    trunks = []
    # Here generate tree data from initial branch
    make_tree(depth, trunk, branch_list_end, trunks, alpha, -np.pi/2)
    pygame.init()
    screen = pygame.display.set_mode(dimensions)
    caption = 'Fractal Tree            '
    caption += '(1)  \'Space\' to start or pause    '
    caption += '(2)  \'esc\' to quit'
    caption += '(3) \'up\' and \'down\' to increase and decrease speed'
    pygame.display.set_caption(caption)
    pygame.display.flip
    
    screen.fill(backgroundColour)
    
    number_of_trunks = len(trunks)
    number_of_end_branches = len(branch_list_end)
    index = 0
    index_end = 0
    ending = False
    draw_branch = False
    keep_running = True
    
    while keep_running:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                keep_running = False
                pygame.quit()
            # Start and pause the animation with the space key 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                draw_branch  = not draw_branch 
            # Speed up animations with up arrow
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                speed_factor = min(speed_factor + 1, 10)
                # Ensure speed_factor is less than or equal to 10
                frames_per_second = 20 + 10 * speed_factor
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                speed_factor = max(speed_factor - 1, 1)
                frames_per_second = 20 + 10 * speed_factor

        # Keep draw next tree with index 'index' if not told to pause and not complete
        if draw_branch and index  < number_of_trunks :
            if thickness:
                pygame.draw.line(screen, brown, trunks[index][0][0],
                            trunks[index][0][1], trunks[index][1])   
            else:
                pygame.draw.line(screen, brown, trunks[index][0][0],
                            trunks[index][0][1], 1)
            # Now update so that latest tree is added 
            pygame.display.update()
            # Pause time before next iteration starts: one clock tick  
            clock.tick(frames_per_second)
            # Index uptate: index walks through trunks indices
            index += 1
        if index == number_of_trunks:
            ending = True
            
        if draw_branch and index_end < number_of_end_branches and ending:
            pygame.draw.line(screen, green, branch_list_end[index_end][0],
                             branch_list_end[index_end][1], 1)
            # Now update so that latest branch is added 
            pygame.display.update()
            # Pause time before next iteration starts: one clock tick  
            clock.tick(frames_per_second)
            # Index uptate: index walks through branch_list_end indices
            index_end += 1
    pygame.quit()
    return None  

Then we can generate different types of tree 

In [ ]:
# NORMAL TREE

draw_tree(depth = 5,alpha = np.pi/6, rat1 = 0.5, rat2 = 1.5, thickness = False)

In [ ]:
# SQUARE TREE

draw_tree(depth = 6,alpha = np.pi/2, rat1 = 0.4, rat2 = 1, 
              thickness = False)

In [ ]:
# TRIANGLE TREE

draw_tree(depth = 9,alpha = 2*np.pi/3, rat1 = 0.4, rat2 = 1, thickness = False)

In [ ]:
# TREE WITH THICKNESS

draw_tree(depth = 6, alpha = np.pi/4, thickness = True)

In [ ]:
draw_tree(depth = 5,alpha = np.pi/6, rat1 = 0.9, rat2 = 1.5,
              thickness = False)


## Reference tree

![treereferenceimage](img/treediagram.png)